In [1]:
import glob
import pickle
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u

/Users/Carlos/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
def f1(x):
    return '%5.2f' % x

In [3]:
cd /Users/Carlos/Documents/HERMES/reductions/6.5/NGC2477/obj/

/Users/Carlos/Documents/HERMES/reductions/6.5/NGC2477/obj


In [5]:
filehandler = open('IFU001.obj', 'r')
thisStar = pickle.load(filehandler)
thisCamera= thisStar.exposures.cameras[0]
thisCamera.fileNames
thisStar.exposures.plates[10][-1]
# print np.vstack((thisStar.exposures.pivots,np.nansum(thisCamera.red_fluxes,axis=1))).transpose()
# print thisStar.exposures.p

IndexError: index 10 is out of bounds for axis 0 with size 1

In [5]:

outFileNames = np.load('/Users/Carlos/Documents/ipn/npy/outFileNames.npy')
# outObjects = np.load('npy/outObjects.npy')
# outPlates = np.load('npy/outPlates.npy')
outMJD_Exp = np.load('/Users/Carlos/Documents/ipn/npy/outMJD_Exp.npy')

# outFileNames = np.delete(outFileNames,86)
# outObjects = np.delete(outObjects,86)
# outPlates = np.delete(outPlates,86)
# outMJD_Exp = np.delete(outMJD_Exp,86, axis = 0)

ordIdx = np.argsort(outMJD_Exp[:,0])

outFileNames = outFileNames[ordIdx]
outMJD_Exp = outMJD_Exp[ordIdx]



In [6]:
obsMatrix = np.ones((292, 99, 2, 4))*np.nan #stars, observations, [pivot, plate], cam

objList = glob.glob('*.obj')

outArray = []
i = 0
for thisObj in objList:
    if 'red' not in thisObj:
        filehandler = open(thisObj, 'r')
        thisStar = pickle.load(filehandler)
#         print thisStar.name
        
        gc = SkyCoord(ra=thisStar.RA*u.si.degree, dec=thisStar.Dec*u.si.degree)
        name = thisStar.name
        mag = thisStar.Vmag
        RAhms = str(int(gc.ra.hms.h))+'h'+str(int(gc.ra.hms.m))+'m'+str(f1(gc.ra.hms.s))+'s'
        RAdms = str(gc.ra)
        RAdeg = str(gc.ra.deg)
        DECdms = str(gc.dec)
        DECdeg = str(gc.dec.deg)
#         pivot = a[1].data['PIVOT'][i]
        i += 1
        ID = i
        outArray.append((ID, name, mag, RAhms, RAdms, RAdeg, DECdms, DECdeg))
        
        for cam in range(4):
#             print 'Camera',cam,
            thisCam = thisStar.exposures.cameras[cam]

            for exp in range(len(thisCam.fileNames)):
                if np.nansum(thisCam.red_fluxes[exp])!=0:
                    thisFileName = thisStar.exposures.cameras[0].fileNames[exp][:10]+'.fits'
#                     print thisFileName
                    fileIdx = np.where(outFileNames==thisFileName)[0]
                    if len(fileIdx)>0:
                        fileIdx = fileIdx[0]
                        obsMatrix[ID-1, fileIdx, 0, cam] = thisStar.exposures.pivots[exp]
                        obsMatrix[ID-1, fileIdx, 1, cam] = thisStar.exposures.plates[exp][-1]
#                         print 'Found', ID, fileIdx, cam
            
            
            
            
        filehandler.close()
        thisStar = None

outArray = np.array(outArray)
np.save('outArray.npy',outArray)

labels = ['ID', 'Name','Vmag','RA(hms)','RA(dms)','RA(deg)','DEC(dms)','DEC(deg)']
df = pd.DataFrame(outArray)
df.columns = labels

outLatex = df.to_latex(index=False)
print df.to_latex(index=False)

\begin{tabular}{llllllll}
\toprule
  ID &        Name &   Vmag &      RA(hms) &        RA(dms) &        RA(deg) &       DEC(dms) &        DEC(deg) \\
\midrule
   1 &      1008.0 &  12.36 &  7h52m18.91s &  118d04m43.65s &  118.078791667 &  -38d31m36.35s &  -38.5267638889 \\
   2 &      1044.0 &  11.83 &  7h52m26.64s &   118d06m39.6s &        118.111 &  -38d29m45.18s &  -38.4958833333 \\
   3 &      1069.0 &   9.81 &  7h52m18.90s &   118d04m43.5s &      118.07875 &   -38d28m22.9s &  -38.4730277778 \\
   4 &      1252.0 &  11.48 &  7h52m30.70s &   118d07m40.5s &  118.127916667 &  -38d27m57.79s &  -38.4660527778 \\
   5 &      2009.0 &  11.01 &  7h52m22.51s &  118d05m37.65s &  118.093791667 &  -38d31m43.16s &  -38.5286555556 \\
   6 &      2036.0 &  12.14 &  7h52m26.90s &   118d06m43.5s &  118.112083333 &   -38d31m47.5s &  -38.5298611111 \\
   7 &      2061.0 &  12.72 &  7h52m41.83s &  118d10m27.45s &  118.174291667 &  -38d32m13.18s &  -38.5369944444 \\
   8 &      2117.0 &  10.04 &  7h52m

In [16]:
np.min(np.array(df['Vmag']).astype(float))

9.8100000000000005

In [ ]:
14.5